# **Modelo automl - Wine Reviews**

In [ ]:
# Instalando pacotes
!pip install pycaret
!pip install umap-learn
!pip install --upgrade nbformat

In [ ]:

# Baixando modelo pré-treinado
!python -m spacy download en_core_web_sm
!python -m textblob.download_corpora

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Biblioteca AutoML Pycaret
from pycaret.classification import *

# Parâmetros de configuração dos gráficos
from matplotlib import rcParams

rcParams['figure.figsize'] = 12, 4
rcParams['lines.linewidth'] = 3
rcParams['xtick.labelsize'] = 'x-large'
rcParams['ytick.labelsize'] = 'x-large'

# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

# Versões dos pacotes usados neste jupyter notebook
#%reload_ext watermark
#%watermark -a "Verções bibliotecas" --iversions

import warnings
warnings.filterwarnings("ignore")

from pycaret.utils import enable_colab
enable_colab()

# **Base dados**

In [ ]:
data = pd.read_csv("/content/winemag-data-130k-v2.csv")
data

In [ ]:
# Visualizando os 5 primeiros dados
data.head(5)

In [ ]:
# Visualizando os 5 últimos dados
data.tail(5)

In [ ]:
# Visualizando linhas e colunas
data.shape

In [ ]:
# Informações dos dados
data.info()

In [ ]:
# Tipo dados
data.dtypes

In [ ]:
# Colunas númericas
nub = ["points", 
       "price"]

# Coluna categorica
cat = ["country", 
       "designation", 
       "province", 
       "region_1",
       "region_2",
       "taster_name",
       "taster_twitter_handle",
       "title",
       "variety",
       "winery"]

# Coluna target
target = ["description"]

In [ ]:
data['description'] = data['description'].astype(str)
data['title'] = data['title'].astype(str)

In [ ]:
# Total description
data["description"].value_counts()

In [ ]:
# Resumo variáveis numéricas
data[nub].describe()

In [ ]:
# Listando o total por categoria de cada variável categórica
for col in cat:
    print(f'''Total de Registros Por Categoria da Variável {col}:''')
    print(data[col].value_counts())
    print()

In [ ]:
features = nub
for i in range(0, len(features)):
    plt.subplot(1, len(features), i + 1)
    sns.boxplot(y = data[features[i]], color = 'blue', orient = 'v')
    plt.tight_layout()

In [ ]:
for i in range(0, len(nub)):
    plt.subplot(1, len(nub), i+1)
    sns.violinplot(y = data[nub[i]], color = 'yellow', orient = 'v')
    plt.tight_layout()

In [ ]:
plt.figure(figsize= (18, 8))
sns.histplot(data["price"])

In [ ]:
from wordcloud import WordCloud

todos_palavras = ' '.join([texto for texto in data["description"]])
nuvem_palavras = WordCloud(width = 800, 
                           height = 500, 
                           max_font_size = 110,
                           collocations = False).generate(todos_palavras)

plt.figure(figsize= (25.5, 17))
plt.imshow(nuvem_palavras, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
todos_palavras = ' '.join([texto for texto in data["title"]])
nuvem_palavras = WordCloud(width = 800, 
                           height = 500, 
                           max_font_size = 110,
                           collocations = False).generate(todos_palavras)

plt.figure(figsize= (25.5, 17))
plt.imshow(nuvem_palavras, interpolation = 'bilinear')
plt.axis('off')
plt.show()

# **Pre-processamento**

In [ ]:
# Bibliotecas sistema
import re
import unicodedata
import itertools

# Importando NLTK NLP
# Baixando pacote do nltk
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Baixando todos os complementos NLTK
nltk.download('punkt')
nltk.download('stopwords')
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
%%time

# Dados de limpeza para modelo PLN
# Variável para treino
x = data["description"]

# Lista de stopwords
list_stop_words = ['covfefe']

# Recebe uma lista de palavras ou texto como entrada e retorna um conjunto de palavras mais limpo. 
# A função faz normalização, codificação/decodificação, letras minúsculas e lematização.
def limpeza(texto):
    WNL = nltk.stem.WordNetLemmatizer()
    stop_words2 = nltk.corpus.stopwords.words("english") + stopwords
    texto = (unicodedata.normalize("NFKD", texto).encode("ascii", "ignore").decode("utf-8", "ignore").lower())
    WORDS = re.sub(r'[^\w\s]', '', texto).split()
    return [WNL.lemmatize(word) for word in WORDS if word not in stop_words2]

# Remove stop words: Removendo as stop words na base de dados
def remove_stop_words(instancia): # Removendo as stop words
    stopwords = set(nltk.corpus.stopwords.words("english"))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# Palavras derivacionalmente relacionadas com significados semelhantes, palavras para retornar documentos que contenham outra palavra no conjunto.
def text_stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
        return (" ".join(palavras))

# Limpeza na base de dados limpando dados de web com http e outros.
def dados_limpos(instancia): 
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

#Lemmatization: Em linguística é o processo de agrupar as formas flexionadas de uma palavra para que possam ser analisadas como um único item, identificado pelo lema da palavra , ou forma de dicionário.
def Lemmatization(instancia):
    palavras = []
    for w in instancia.split():
        palavras.append(wordnet_lemmatizer.lemmatize(w))
        return (" ".join(palavras))

# Preprocessing: Pré - processamento da base de dados que serão ser para análise de dados.
def Preprocessing(instancia):
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','')
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# remover caracteres alphanuméricos
alphanumeric = lambda x: re.sub('\w*\d\w*', '', x)

# remover pontuações
punc = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x)

# remover espaços duplos
remove_space = lambda x: x.replace('  ', ' ')

# Removendo pontuações descrições itens com expressão regular
data['description'] = data['description'].str.replace('[,.:;!?]+', ' ', regex=True).copy()

# Removendo os caracteres especiais de descrições 
data['description'] = data['description'].str.replace ('[/<>()|\+\-\$%&#@\'\"]+', ' ', regex=True).copy()

# Removendo palavras com números 
data['description'] = data['description'].str.replace('[0-9]+', '', regex=True).copy()

# Gerando a base dados limpo
x = [Preprocessing(i) for i in x]

# Aplicando nova coluna
data["text_clean"] = data["description"].apply(Preprocessing)

# Mundando os tipo de dados de object para string 
data["text_clean"] = data["text_clean"].astype(str)

# Salvando base textos
# Salvando arquivo em csv
data.to_csv("text_clean.csv", index=False)

# Salvando arquivo em text
data.to_csv("text_clean.text", index=False)

In [ ]:
# Visualizando coluna texto
data.text_clean.head(15)

In [ ]:
# Nuvem de palavras coluna texto limpo
todos_palavras = ' '.join([texto for texto in data["text_clean"]])
nuvem_palavras = WordCloud(width = 800, 
                           height = 500, 
                           max_font_size = 110,
                           collocations = False).generate(todos_palavras)

plt.figure(figsize= (25.5, 17))
plt.title("Nuvem de palavras limpas")
plt.imshow(nuvem_palavras, interpolation = 'bilinear')
plt.axis('off')
plt.show()

# **Modelo nlp - Pré treinado spacy**

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

# **Modelo NLP - AutoML**

In [ ]:
from pycaret.nlp import *

model = setup(data = data, 
                   target = 'text_clean', 
                   session_id = 123)

In [ ]:
lda = create_model('lda')
print(lda)

In [ ]:
lda2 = create_model('lda', 
                    num_topics = 10, 
                    multi_core = True)
print(lda2)

In [ ]:
lda_results = assign_model(lda)
lda_results.head()

In [ ]:
lda_results.to_csv("results.csv")

In [ ]:
plot_model()

In [ ]:
plot_model(lda, plot = 'frequency')

In [ ]:
plot_model(lda, plot = 'topic_distribution')

In [ ]:
plot_model(lda, plot = 'topic_model')

In [ ]:
plot_model(plot = 'bigram')

In [ ]:
plot_model(lda, plot = 'frequency', topic_num = 'Topic 1')

In [ ]:
plot_model(lda, plot = 'frequency', topic_num = 'Topic 2')

In [ ]:
plot_model(lda, plot = 'topic_distribution')

In [ ]:
evaluate_model(lda)

# **Modelo 2 Automl**

In [ ]:
stopwords = ['co','http','go','get']
setup(data, 
      target = 'description',
      session_id = 123, 
      custom_stopwords = stopwords)

In [ ]:
lda2 = create_model('lda')

In [ ]:
plot_model(lda2, plot = 'topic_distribution')

In [ ]:
df2 = assign_model(lda2)
df2

# **Tune model**

In [ ]:
exp_nlp102 = setup(data = data, 
                   target = 'text_clean', 
                   session_id = 123,
                   custom_stopwords = ['loan', 'income', 'usd', 'many', 'also', 'make', 'business', 'buy', 
                                       'sell', 'purchase','year', 'people', 'able', 'enable', 'old', 'woman',
                                       'child', 'school'],
                   log_experiment = True)

In [ ]:
lda = create_model('lda')
lda

In [ ]:
plot_model(lda, plot = 'topic_distribution')

In [ ]:
tuned_unsupervised = tune_model(model = 'lda', multi_core = True)
tuned_unsupervised

In [ ]:
save_model(lda,'Final LDA Model 08Feb2020')

In [ ]:
saved_lda = load_model('Final LDA Model 08Feb2020')
print(saved_lda)